# Set seed and logger

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import logging
import os
import random
import re
import numpy as np
import torch

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    random.seed(seed)
    
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

def set_global_logging_level(level=logging.ERROR, prefixes=("",)):
    """
    Override logging levels of different modules based on their name as a prefix.
    It needs to be invoked after the modules have been loaded so that their loggers have been initialized.

    Args:
        level: desired level. Optional. Default is logging.ERROR
        prefixes: list of one or more str prefixes to match (e.g. ["transformers", "torch"]). Optional.
            Default is `[""]` to match all active loggers.
            The match is a case-sensitive `module_name.startswith(prefix)`
    """
    prefix_re = re.compile(fr'^(?:{"|".join(prefixes)})')
    for name in logging.root.manager.loggerDict:
        if re.match(prefix_re, name):
            logging.getLogger(name).setLevel(level)

In [3]:
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# if logger.hasHandlers():
#     logger.handlers.clear()
# console = logging.StreamHandler()
# logger.addHandler(console)

logging.basicConfig(
    format='[%(asctime)s %(levelname)s %(name)s] %(message)s', datefmt='%m/%d %H:%M:%S',
    level=logging.INFO
)
set_global_logging_level(logging.WARNING, ["elasticsearch"])

set_seed(0)

# Load examples

In [4]:
import json

split = 'dev'
samples = []
with open(f"data/hotpot-{split}.tsv") as f:
    for line in f:
        if split == 'test':
            q_id, question = line.strip().split('\t')
            samples.append((q_id, (question,)))
        else:
            q_id, question, answer, sp_facts = line.strip().split('\t')
            sp_facts = json.loads(sp_facts)
            samples.append((q_id, (question, answer, sp_facts)))
print(len(samples))

7405


In [5]:
if split == 'test':
    gold_qas_map = None
    gold_samples = None
    gold_state2action = None
else:
    gold_qas_map = dict(samples)
    with open('data/HotpotQA/hotpot_dev_fullwiki_v1.json') as f:
        gold_samples = json.load(f)
    print(len(gold_samples))
    gold_state2action = dict()
    with open('data/hotpot-step-dev.strict.refined.jsonl') as f:
        for line in f:
            example = json.loads(line)
            gold_state2action[example['_id']] = example['state2action']
    print(len(gold_state2action))

7405
7405


# Load sparse query generator

In [6]:
from retriever import SparseRetriever

sparse_retriever = SparseRetriever('enwiki-20171001-paragraph-5', ['10.60.0.59:9200'], max_retries=3, timeout=30)

[06/11 00:45:32 INFO transformers.file_utils] PyTorch version 1.4.0 available.
[06/11 00:45:33 INFO faiss.loader] Loading faiss with AVX2 support.
[06/11 00:45:33 INFO faiss.loader] Loading faiss.


In [7]:
# %env CLASSPATH=corenlp/*
from drqa.reader import Predictor
import warnings

warnings.filterwarnings("ignore")

# import os
# os.environ['CLASSPATH'] = 'corenlp/*'

qg1 = Predictor(model='ckpts/golden-retriever/hop1.mdl', tokenizer=None, embedding_file='data/glove.840B.300d.txt', num_workers=-1)
qg1.cuda()
qg1.model.network.to(torch.device('cuda:0'))
qg2 = Predictor(model='ckpts/golden-retriever/hop2.mdl', tokenizer=None, embedding_file='data/glove.840B.300d.txt', num_workers=-1)
qg2.cuda()
qg2.model.network.to(torch.device('cuda:0'))

[06/11 00:45:34 INFO drqa.reader.predictor] Initializing model...
[06/11 00:45:34 INFO drqa.reader.model] Loading model ckpts/golden-retriever/hop1.mdl
[06/11 00:45:34 INFO drqa.reader.predictor] Expanding dictionary...
[06/11 00:46:22 INFO drqa.reader.model] Adding 2115762 new words to dictionary...
[06/11 00:46:25 INFO drqa.reader.model] New vocab size: 2195963
[06/11 00:46:35 INFO drqa.reader.model] Loading pre-trained embeddings for 2115762 words from data/glove.840B.300d.txt
[06/11 00:46:45 WARNING root] WARN: Duplicate embedding found for ����������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������������
[06/11 00:46:51 WARNING root] WARN: Duplicate embedding found for Kṛṣṇa
[06/11 00:47:26 WARNING root] WARN: Dup

RnnDocReader(
  (embedding): Embedding(2195963, 300, padding_idx=0)
  (qemb_match): SeqAttnMatch(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (doc_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(662, 128, bidirectional=True)
      (1): LSTM(256, 128, bidirectional=True)
      (2): LSTM(256, 128, bidirectional=True)
    )
  )
  (question_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 128, bidirectional=True)
      (1): LSTM(256, 128, bidirectional=True)
      (2): LSTM(256, 128, bidirectional=True)
    )
  )
  (self_attn): LinearSeqAttn(
    (linear): Linear(in_features=768, out_features=1, bias=True)
  )
  (start_attn): BilinearSeqAttn(
    (linear): Linear(in_features=768, out_features=768, bias=True)
  )
  (end_attn): BilinearSeqAttn(
    (linear): Linear(in_features=768, out_features=768, bias=True)
  )
)

# Load dense indexer and dense encoder

In [8]:
import faiss
faiss.omp_set_num_threads(16)

In [9]:
from argparse import Namespace

args = Namespace(**{
    "model_name": "roberta-base",
    "model_path": "ckpts/mdr/q_encoder.pt",
    "index_prefix_path": "data/index/mdr/hotpot-paragraph-q-strict",  # .hnsw
    "index_buffer_size": 50000,
    "max_q_len": 70,
    "max_q_sp_len": 350
})

In [10]:
from transformers import AutoConfig, AutoTokenizer
from mdr.retrieval.models.retriever import RobertaCtxEncoder
from utils.model_utils import load_state

bert_config = AutoConfig.from_pretrained(args.model_name)
dense_tokenizer = AutoTokenizer.from_pretrained(args.model_name)
dense_encoder = RobertaCtxEncoder(bert_config, args)
dense_encoder = load_state(dense_encoder, args.model_path, exact=False)
dense_encoder.to(torch.device('cuda:0'))
dense_encoder.eval()

[06/11 00:55:46 INFO transformers.configuration_utils] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /home/shenhuawei/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
[06/11 00:55:46 INFO transformers.configuration_utils] Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

[06/11 00:55:48 INFO transformers.configuration_utils] 

RobertaCtxEncoder(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
from dense_indexers import DenseHNSWFlatIndexer, DenseFlatIndexer

vector_size = bert_config.hidden_size
if args.index_prefix_path.endswith('hnsw'):
    dense_indexer = DenseHNSWFlatIndexer(vector_size, args.index_buffer_size)
else:
    dense_indexer = DenseFlatIndexer(vector_size, args.index_buffer_size)
dense_indexer.deserialize_from(args.index_prefix_path)

[06/11 00:56:06 INFO dense_indexers] Loading index from data/index/mdr/hotpot-paragraph-q-strict
[06/11 00:56:40 INFO dense_indexers] Loaded index of type <class 'faiss.swigfaiss.IndexFlat'> and size 5232080


In [12]:
from retriever import DenseRetriever

dense_retriever = DenseRetriever(dense_indexer, dense_encoder, dense_tokenizer)

# Load corpus

In [13]:
from utils.data_utils import load_corpus

corpus, title2id = load_corpus('data/corpus/hotpot-paragraph-5.tsv', for_hotpot=True, require_hyperlinks=True)
print(len(corpus))
print(len(title2id))

[06/11 01:01:42 INFO utils.data_utils] Loaded 5232080 passages from data/corpus/hotpot-paragraph-5.tsv


5232080
5232080


# Load union model

In [14]:
from models.union_model import UnionModel

tokenizer = AutoTokenizer.from_pretrained('google/electra-base-discriminator', use_fast=True,
                                           additional_special_tokens=['[unused0]', '[unused1]',
                                                                      '[unused2]', '[unused3]'])
union_model = UnionModel('google/electra-base-discriminator', max_ans_len=64)

[06/11 01:01:43 INFO transformers.configuration_utils] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/google/electra-base-discriminator/config.json from cache at /home/shenhuawei/.cache/torch/transformers/9236d197566a7f1be2b2151f5afcc5a8e17f31e1e23c52f3cdf2340019986e78.88ba6e8e7d5a7936e86d6f2551fe19c236dc57c24da163907cd0544e9933f6ee
[06/11 01:01:43 INFO transformers.configuration_utils] Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "type_vocab_

In [15]:
init_checkpoint = 'ckpts/td5-exp1-ila.4_electra-base-discriminator_DP0.5_HN2_M2_D2_adamW_SP0.5_B32_LR2.0e-05_WU0.1_E30_S42_04202303_pld-sb0-wo*-cmd10/checkpoint_68000.pt'
device = torch.device("cuda:0")

union_model = load_state(union_model, init_checkpoint)
union_model.to(device)
union_model.eval()

UnionModel(
  (encoder): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

# Env: WikiWorld

In [16]:
import redis
from tqdm.auto import tqdm
from env.core import Environment

bm25_redis = redis.Redis(host='10.60.1.79', port=6379, db=3, password='redis4zyc', decode_responses=True)
mdr_redis = redis.Redis(host='10.60.1.79', port=6379, db=4, password='redis4zyc', decode_responses=True)
env = Environment(corpus, title2id, sparse_retriever, dense_retriever, bm25_redis, mdr_redis,
                  for_hotpot=True, strict=True, max_ret_size=1000)

# QA agent

In [25]:
import copy
from agent import Agent
from hotpot_evaluate_plus import evaluate, pretty_metrics

faiss.omp_set_num_threads(1)
query_redis = redis.Redis(host='10.60.1.79', port=6379, db=2, password='redis4zyc', decode_responses=True)
agent = Agent(tokenizer, union_model, qg1, qg2, device, env, query_redis,
              actions=("ANSWER", "BM25", "MDR", "LINK"), action_mask=(1, 1, 1, 1),
              memory_size=2, max_seq_len=512, max_q_len=96, max_obs_len=256, strict=True,
              gold_qas_map=gold_qas_map, oracle_belief=False, oracle_state2action=None)  # gold_state2action

In [26]:
predictions = {"answer": dict(), "norm_answer": dict(), "sp": dict(), "_sp": dict(), "spp": dict()}
n_observed = -1
env.reset()
agent.reset()
agent.eval()
q_ids = [sample[0] for sample in samples]
questions = [sample[1][0] for sample in samples]
observations = [None] * len(q_ids)

In [27]:
def force_answer():
    if split != 'test':
        answers, _answers, sp_passages, sp_facts, _sp_facts = agent.force_answer(q_ids, questions)
        tmp_predictions = copy.deepcopy(predictions)
        tmp_predictions['answer'].update(answers)
        tmp_predictions['norm_answer'].update(_answers)
        tmp_predictions['sp'].update(sp_facts)
        tmp_predictions['_sp'].update(_sp_facts)
        tmp_predictions['spp'].update(sp_passages)
        print(pretty_metrics(evaluate(tmp_predictions, gold_samples)))
        del tmp_predictions

In [28]:
MAX_OBS = 10
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx observed memory
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed > 0 and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

gen_sparse_query:   0%|          | 0/7405 [00:00<?, ?it/s]

act:   0%|          | 0/232 [00:00<?, ?it/s]

exe:   0%|          | 0/7405 [00:00<?, ?it/s]

0: 7405 remained


gen_sparse_query:   0%|          | 0/7405 [00:00<?, ?it/s]

...

gen_sparse_query:   0%|          | 0/748 [00:00<?, ?it/s]

act:   0%|          | 0/24 [00:00<?, ?it/s]

[06/11 01:12:52 INFO agent] 5a7d19d85542995ed0d165e8: augment SP ['35001974_0'] into memory
[06/11 01:12:52 WARNING agent] 5a7d19d85542995ed0d165e8: augment false SP 35001974_0 into memory {'79371_0': 0.8471337556838989, '35001974_0': 0.29022565484046936}
[06/11 01:12:53 INFO agent] 5a7363ec5542991f29ee2dd7: miss SP1 109495_0 (Key West, Florida) for 0.0404 <= 0.5634 when 381 tokens
[06/11 01:12:54 INFO agent] 5ae75a5a5542991bbc9761f3: miss SP1 35121149_0 (Raabta (song)) for 0.2161 <= 0.6567 when 366 tokens
[06/11 01:12:55 INFO agent] 5ae5e762554299546bf82faf: miss SP1 2110323_0 (Rihanna) for 0.4111 <= 0.6750 when 309 tokens
[06/11 01:12:55 INFO agent] 5ae5e762554299546bf82faf: augment SP ['2110323_0'] into memory
[06/11 01:12:56 INFO agent] 5ac17b485542994ab5c67d65: miss SP0 23786794_0 (St Wilfrid's Church, Alford) for 0.0555 <= 0.6298 when 302 tokens
[06/11 01:12:57 INFO agent] 5a80b9ae5542992bc0c4a7eb: augment SP ['14900848_0'] into memory
[06/11 01:12:57 WARNING agent] 5a80b9ae55429

exe:   0%|          | 0/748 [00:00<?, ?it/s]

10: 721 remained
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 59.43 | 71.34 | 73.77 |  72.54 |
|      Norm answer | 58.87 | 70.75 | 73.24 |  71.89 |
| Support sentence | 52.69 | 76.55 | 79.01 |  76.03 |
| Support sentence | 50.99 | 76.08 | 79.81 |  75.03 |
|  Support passage | 81.36 | 85.13 | 85.13 |  85.13 |
|            Joint | 38.43 | 63.13 | 67.24 |  63.70 |
3.236866981769075
|                  |    EM |    F1 |  Prec | Recall |
|------------------|-------|-------|-------|--------|
|           Answer | 60.90 | 73.44 | 75.94 |  74.74 |
|      Norm answer | 60.34 | 72.87 | 75.45 |  74.09 |
| Support sentence | 53.10 | 79.06 | 81.74 |  78.43 |
| Support sentence | 51.21 | 78.80 | 83.72 |  77.25 |
|  Support passage | 82.16 | 88.05 | 88.05 |  88.05 |
|            Joint | 38.65 | 64.17 | 68.38 |  64.74 |


In [ ]:
assert False

In [ ]:
MAX_OBS = 20
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 5 == 0 and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

In [ ]:
MAX_OBS = 30
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 5 == 0  and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

In [ ]:
MAX_OBS = 50
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 10 == 0  and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

In [ ]:
MAX_OBS = 100
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

while len(observations) > 0 and n_observed < MAX_OBS:
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in tqdm(zip(q_ids, questions), desc='exe', total=len(q_ids)):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    print(f"{n_observed}: {len(observations)} remained")
    if n_observed % 10 == 0 and split != 'test':
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()

In [ ]:
MAX_OBS = 300
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

In [ ]:
MAX_OBS = 500
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

In [ ]:
MAX_OBS = 700
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

In [ ]:
MAX_OBS = 1000
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0, disable_tqdm=True)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 50 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

In [ ]:
assert False

In [ ]:
MAX_OBS = 1500
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 100 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

In [ ]:
MAX_OBS = 2000
set_global_logging_level(logging.WARNING, ["elasticsearch", "dpr"])

pbar = tqdm(total=MAX_OBS - n_observed)
while len(observations) > 0 and n_observed < MAX_OBS:
    pbar.set_description(f"act - {len(observations)}")
    commands = agent.act(q_ids, questions, observations, review=(n_observed + 1) % 100 == 0)
    pbar.set_description(f"exe - {len(observations)}")
    _q_ids, _questions, _observations = [], [], []
    for q_id, question in zip(q_ids, questions):
        cmd = commands[q_id]
        if cmd[0] == 'ANSWER':
            predictions['answer'][q_id] = agent.answer[q_id]
            predictions['norm_answer'][q_id] = max(agent.all_answer[q_id].keys(), key=lambda k: agent.all_answer[q_id][k]) if len(agent.all_answer[q_id]) > 0 else 'noanswer'
            predictions['sp'][q_id] = list(agent.sp_facts[q_id].keys())
            predictions['_sp'][q_id] = list(agent.all_sp_facts[q_id].keys())
            predictions['spp'][q_id] = agent.sp_passages[q_id]
            continue
        else:
            obs = env.step(cmd, q_id, exclusion=agent.memory[q_id])  # xxx agent.observed[q_id]
            _q_ids.append(q_id)
            _questions.append(question)
            _observations.append(obs)
    q_ids, questions, observations = _q_ids, _questions, _observations
    n_observed += 1
    pbar.update(1)
    if n_observed % 100 == 0 and split != 'test':
        print(f"{n_observed}: {len(observations)} remained")
        print(pretty_metrics(evaluate(predictions, gold_samples)))
        print(sum([len(x) for x in agent.observed.values()]) / len(agent.observed))
force_answer()
pbar.close()

# Analyze

## Error types

In [ ]:
print({k: len(v) for k, v in agent.cases.items()})

### **ans none and early ans**

In [ ]:
print(len(agent.cases['ans_none']), len(agent.cases['ans_none'] & agent.cases['early_ans']), len(agent.cases['early_ans']))

In [ ]:
print(len(agent.cases['ans_none'] - agent.cases['good_spp']), len(agent.cases['ans_none'] & agent.cases['good_spp']), len(agent.cases['ans_none']))
print(len(agent.cases['ans_none'] - agent.cases['good_ans']), len(agent.cases['ans_none'] & agent.cases['good_ans']), len(agent.cases['ans_none']))

In [ ]:
print(len(agent.cases['early_ans'] - agent.cases['good_spp']), len(agent.cases['early_ans'] & agent.cases['good_spp']), len(agent.cases['early_ans']))
print(len(agent.cases['early_ans'] - agent.cases['good_ans']), len(agent.cases['early_ans'] & agent.cases['good_ans']), len(agent.cases['early_ans']))

### false final sp sentences

In [ ]:
print(len(agent.cases['bad_sp'] - agent.cases['good_spp']), len(agent.cases['bad_sp'] & agent.cases['good_spp']), len(agent.cases['bad_sp']))
print(len(agent.cases['bad_sp'] - agent.cases['good_ans']), len(agent.cases['bad_sp'] & agent.cases['good_ans']), len(agent.cases['bad_sp']))

### **false final sp passages**

In [ ]:
bad_spp_cases = set(agent.observed.keys()) - agent.cases['good_spp']

In [ ]:
print(len(bad_spp_cases - agent.cases['good_ans']), len(bad_spp_cases & agent.cases['good_ans']), len(bad_spp_cases))

In [ ]:
from collections import Counter

rec_spp_nums = [len(set(agent.pretty_memory(q_id).keys()) & set(gold_qas_map[q_id][2].keys())) for q_id in bad_spp_cases]

print(Counter(rec_spp_nums))

### **add distractor into memory**

In [ ]:
print(len(agent.cases['add_dis'] - agent.cases['good_spp']), len(agent.cases['add_dis'] & agent.cases['good_spp']), len(agent.cases['add_dis']))
print(len(agent.cases['add_dis'] - agent.cases['good_ans']), len(agent.cases['add_dis'] & agent.cases['good_ans']), len(agent.cases['add_dis']))

### **miss sp passage**

In [ ]:
print(len(agent.cases['miss_sp'] - agent.cases['good_spp']), len(agent.cases['miss_sp'] & agent.cases['good_spp']), len(agent.cases['miss_sp']))
print(len(agent.cases['miss_sp'] - agent.cases['good_ans']), len(agent.cases['miss_sp'] & agent.cases['good_ans']), len(agent.cases['miss_sp']))

### miss 2nd sp passage

In [ ]:
print(len(agent.cases['miss_sp2'] - agent.cases['good_spp']), len(agent.cases['miss_sp2'] & agent.cases['good_spp']), len(agent.cases['miss_sp2']))
print(len(agent.cases['miss_sp2'] - agent.cases['good_ans']), len(agent.cases['miss_sp2'] & agent.cases['good_ans']), len(agent.cases['miss_sp2']))

### remove sp

In [ ]:
print(len(agent.cases['rm_sp'] - agent.cases['good_spp']), len(agent.cases['rm_sp'] & agent.cases['good_spp']), len(agent.cases['rm_sp']))
print(len(agent.cases['rm_sp'] - agent.cases['good_ans']), len(agent.cases['rm_sp'] & agent.cases['good_ans']), len(agent.cases['rm_sp']))

In [ ]:
print(len(agent.cases['rm_sp'] & agent.cases['add_dis']), len(agent.cases['rm_sp'] - agent.cases['add_dis']), len(agent.cases['rm_sp']))
print(len(agent.cases['rm_sp'] & agent.cases['add_dis'] & agent.cases['good_spp']), len(agent.cases['rm_sp'] & agent.cases['add_dis'] - agent.cases['good_spp']))
print(len(agent.cases['rm_sp'] - agent.cases['add_dis'] & agent.cases['good_spp']), len(agent.cases['rm_sp'] - agent.cases['add_dis'] - agent.cases['good_spp']))

### **false dense query expansion**

In [ ]:
print(len(agent.cases['false_expansion'] - agent.cases['good_spp']), len(agent.cases['false_expansion'] & agent.cases['good_spp']), len(agent.cases['false_expansion']))
print(len(agent.cases['false_expansion'] - agent.cases['good_ans']), len(agent.cases['false_expansion'] & agent.cases['good_ans']), len(agent.cases['false_expansion']))

In [ ]:
print(len(agent.cases['false_expansion'] - agent.cases['add_dis']), len(agent.cases['false_expansion']), len(agent.cases['false_expansion'] & agent.cases['add_dis']), len(agent.cases['add_dis']), len(agent.cases['add_dis'] - agent.cases['false_expansion']))

## final evidence set size

In [ ]:
from collections import defaultdict

msz2cases = defaultdict(set)
for q_id, mem in agent.memory.items():
    msz2cases[len(mem)].add(q_id)

for msz in sorted(msz2cases.keys()):
    cases = msz2cases[msz]
    print(f"{msz}: {len(cases)}")
    print(len(cases - agent.cases['good_spp']), len(cases & agent.cases['good_spp']), len(cases - agent.cases['good_ans']), len(cases & agent.cases['good_ans']))
    print()

## case studty

In [ ]:
for x in agent.exhausted_cmds.values():
    if len(x) > 0:
        print(x)

In [ ]:
cnt = 0
for k, v in agent.memory.items():
    if len(v) >= 3 and k in predictions['answer']:
        if cnt == 0:
            print(k, v)
        cnt += 1
print(cnt)

In [ ]:
q_id = '5a7149595542994082a3e76f'  # 5ab3abe9554299753aec597e 5a7363ec5542991f29ee2dd7 5ae54bbd5542990ba0bbb262 5adf29f05542993344016c09 5a764c0b55429976ec32bd89
if q_id in predictions['answer']:
    print(predictions['answer'][q_id])
print(agent.all_answer[q_id])
if q_id in predictions['sp']:
    print(predictions['sp'][q_id])
print(agent.all_sp_facts[q_id])
if q_id in predictions['spp']:
    print(predictions['spp'][q_id])
print(agent.pretty_memory(q_id))

In [ ]:
gold_qas_map[q_id]

In [ ]:
print(agent.pretty_behavior(q_id))

In [ ]:
print(f"*{0:<3d} Q\t{agent.pretty_cmd(agent.commands[q_id][0])}")
for i, (obs, obs_score, cmd, proposals) in enumerate(zip(agent.ids2titles(agent.observed[q_id]), agent.obs_scores[q_id], agent.commands[q_id][1:], agent.proposals[q_id][1:])):
    print(f"*{i + 1:<3d} {obs_score:.3f} {obs}\t{agent.pretty_cmd(cmd)}")

In [ ]:
print(f"*{0:<3d} Q    {agent.pretty_cmd(agent.commands[q_id][0])}")
for cmd, conf, prob in agent.proposals[q_id][0]:
    print(f"{prob:.3f}  {conf:.2f}  {agent.pretty_cmd(cmd)}")
for i, (obs, obs_score, cmd, proposals) in enumerate(zip(agent.ids2titles(agent.observed[q_id]), agent.obs_scores[q_id], agent.commands[q_id][1:], agent.proposals[q_id][1:])):
    print(f"*{i + 1:<3d} {obs_score:.3f} -- {obs}    {agent.pretty_cmd(cmd)}")
    for cmd, conf, prob in proposals:
        print(f"{prob:.3f}  {conf:.2f} {agent.pretty_cmd(cmd)}")

In [ ]:
agent.clicked[q_id]

In [ ]:
agent.observed[q_id]

In [ ]:
agent.obs_scores[q_id]